In [1]:
import pandas as pd
import transformers 
import numpy as np
import torch
import sys

sys.path.insert(0, '..')
from decompose_roberta import RobertaForMaskedLMDecomposed


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilroberta-base")

decomposed_model = RobertaForMaskedLMDecomposed.from_pretrained(
    "distilbert/distilroberta-base",
    debug=False
    )


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
verbs = pd.read_csv("all_VERBs.csv")["WORD"]
verb_ids = []

for verb in verbs.iloc:
    i = tokenizer.encode(" " + verb, add_special_tokens=False)
    if (len(i) == 1):
        verb_ids.append(i[0])

verb_ids = torch.Tensor(verb_ids).to(int)


In [4]:
number_df = pd.read_csv("nounpp.tsv", delimiter="\t")
max_length = 16

number_df["id"] = number_df["id"].apply(lambda x: int(x[2:]))

# only keep sentences that are plural/singular or singular/plural (distractor has different number)
number_df["subject_distractor_number"] = number_df["subject_distractor_number"].apply(
    lambda x: x if x == "singular_plural" or x == "plural_singular" else np.nan
)
number_df.dropna(inplace=True)

number_df["subject_number"] = number_df["subject_distractor_number"].apply(lambda x: x.split("_")[0])
number_df["distractor_number"] = number_df["subject_distractor_number"].apply(lambda x: x.split("_")[1])

number_df["verb"] = number_df["sentence"].apply(lambda x: " " + x.split(" ")[-1])
number_df["sentence"] = number_df["sentence"].apply(lambda x: " ".join(x.split(" ")[:-1]) + "<mask>")

number_df = number_df.drop(
    columns=["subject_distractor_number"]
    ).pivot(index=["id", "subject_number", "distractor_number", "sentence"], columns=["correctness"], values=["verb"]).reset_index()


def get_token(correct_token, wrong_token):
    # drop if longer than 1 token
    if len(correct_token) > 3 or len(wrong_token) > 3:
        return np.nan

    return [correct_token[1], wrong_token[1]]

number_df[("token", "correct")] = tokenizer(number_df[("verb", "correct")].to_list())["input_ids"]
number_df[("token", "wrong")] = tokenizer(number_df[("verb", "wrong")].to_list())["input_ids"]

number_df[("verb_tokens")] = number_df.apply(
    lambda x: get_token(x[("token", "correct")], x[("token", "wrong")]),
    axis=1)

number_df[("sentence_tokens")] = tokenizer(number_df[("sentence", "")].to_list(),
                                           max_length=max_length,
                                           padding="max_length")["input_ids"]

# drop duplicates
number_df = number_df.dropna().reset_index(drop=True).drop(columns="token")
number_df


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_94035/517341255.py:42: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  number_df = number_df.dropna().reset_index(drop=True).drop(columns="token")


id subject_number distractor_number  \
correctness                                          
0             601       singular            plural   
1             602       singular            plural   
2             603       singular            plural   
3             604       singular            plural   
4             605       singular            plural   
..            ...            ...               ...   
882          1795         plural          singular   
883          1796         plural          singular   
884          1797         plural          singular   
885          1799         plural          singular   
886          1800         plural          singular   

                                        sentence         verb                \
correctness                                           correct         wrong   
0             The athlete behind the bikes<mask>     approves       approve   
1              The athlete behind the cars<mask>     inspires       inspire   
2              The athlete behind the cats<mask>      engages        engage   
3              The athlete behind the cats<mask>    remembers      remember   
4            The athlete behind the chairs<mask>     observes       observe   
..                                           ...          ...           ...   
882                 The women near the dog<mask>       engage       engages   
883                The women near the tree<mask>        greet        greets   
884              The women near the window<mask>        avoid        avoids   
885              The women near the window<mask>       engage       engages   
886              The women near the window<mask>   understand   understands   

                verb_tokens                                    sentence_tokens  
correctness                                                                     
0             [28634, 7244]  [0, 133, 8340, 639, 5, 12119, 50264, 2, 1, 1, ...  
1             [25362, 9769]  [0, 133, 8340, 639, 5, 1677, 50264, 2, 1, 1, 1...  
2             [21549, 4949]  [0, 133, 8340, 639, 5, 10017, 50264, 2, 1, 1, ...  
3             [12621, 2145]  [0, 133, 8340, 639, 5, 10017, 50264, 2, 1, 1, ...  
4            [34649, 14095]  [0, 133, 8340, 639, 5, 10826, 50264, 2, 1, 1, ...  
..                      ...                                                ...  
882           [4949, 21549]  [0, 133, 390, 583, 5, 2335, 50264, 2, 1, 1, 1,...  
883          [17395, 33521]  [0, 133, 390, 583, 5, 3907, 50264, 2, 1, 1, 1,...  
884           [1877, 29879]  [0, 133, 390, 583, 5, 2931, 50264, 2, 1, 1, 1,...  
885           [4949, 21549]  [0, 133, 390, 583, 5, 2931, 50264, 2, 1, 1, 1,...  
886            [1346, 8832]  [0, 133, 390, 583, 5, 2931, 50264, 2, 1, 1, 1,...  

[887 rows x 8 columns]

In [7]:
def make_component_masks(sentence_tokens):
    # prep = [639, 13276, 583] # beside, near, behind
    prep = tokenizer.encode(' behind beside near')[1:-1]

    # always starts with 0, 133
    i = 2
    init_i = 0
    The_i = 1
    subj_i = []

    while sentence_tokens[i] not in prep:
        subj_i.append(i)
        i += 1
        
    prep_i = i
    the_i = i + 1
    distractor_id = []
    i += 2

    while sentence_tokens[i] != tokenizer.mask_token_id:
        distractor_id.append(i)
        i += 1
    
    mask_i = i
    
    # place each one in a separate array 
    component_masks = np.zeros((6, max_length))
    for n, component in enumerate([
        init_i, The_i, subj_i, prep_i, the_i, distractor_id
    ]):
        component_masks[n, component] = 1
    
    if mask_i != 6:
        print(sentence_tokens)
        print(component_masks)
        print(mask_i)
        print()
    return component_masks, mask_i


number_df["beta_mask"] = number_df["sentence_tokens"].apply(make_component_masks)


[0, 133, 512, 40381, 639, 5, 12119, 50264, 2, 1, 1, 1, 1, 1, 1, 1]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
7

[0, 133, 512, 40381, 639, 5, 10017, 50264, 2, 1, 1, 1, 1, 1, 1, 1]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
7

[0, 133, 512, 40381, 639, 5, 10826, 50264, 2, 1, 1, 1, 1, 1, 1, 1]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 

In [8]:
def get_proportion_contribution(n):
    beta_masks = torch.tensor(number_df["beta_mask"][n][0])
    verb_tokens = torch.tensor(number_df["verb_tokens"][n])
    inputs = torch.tensor(number_df["sentence_tokens"][n]).unsqueeze(0)
    mask_i = number_df["beta_mask"][n][1]

    contribution_logits = torch.zeros((6, 2))
    # print(tokenizer.convert_ids_to_tokens(verb_tokens))

    for i, mask in enumerate(beta_masks):
        beta_mask = torch.stack([mask, 1 - mask]).unsqueeze(0)

        with torch.no_grad():
            result = decomposed_model(input_ids=inputs,
                            beta_mask=beta_mask,
                            num_contributions=2)
        result = result[0, 0, mask_i, :]

        # normalize across all verbs
        result = result - result[verb_ids].mean()
        result = result / result[verb_ids].std()

        # logits = result[verb_tokens]
        # logits = result
        # beta_z_t / z_t
        # logits = decomp_activation(logits, torch.sigmoid)
        contribution = result[verb_tokens]
        contribution_logits[i, :] = contribution

    return contribution_logits


In [9]:
contribution_logits = []
for i in range(100):
    contribution_logits.append(get_proportion_contribution(i))
    if i % 20 == 0:
        print(i)

contribution_logits = torch.stack(contribution_logits)


0
20
40
60
80


In [10]:
contribution_logits.mean(0)


tensor([[-0.3083, -0.6926],
        [ 0.1906, -0.2531],
        [ 0.3448, -0.0927],
        [-0.5810, -0.6184],
        [-0.7969, -0.7096],
        [-1.5648,  0.2274]])

In [11]:
contribution_logits.mean(0)


tensor([[-0.3782, -0.7139],
        [ 0.7009,  0.0661],
        [-0.1438,  0.2958],
        [ 0.2284, -0.0584],
        [ 0.1192,  0.0749],
        [ 0.3291,  0.4360]])

In [11]:
contribution_logits_sp = []
contribution_logits_ps = []

for i in range(len(number_df)):
    if number_df["subject_number"][i] == "singular":
        contribution_logits_sp.append(get_proportion_contribution(i))
    else:
        contribution_logits_ps.append(get_proportion_contribution(i))
    if i % 20 == 0:
        print(i)

contribution_logits_sp = torch.stack(contribution_logits_sp)
contribution_logits_ps = torch.stack(contribution_logits_ps)


0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880


In [14]:
contribution_logits_sp.mean(0)


tensor([[-0.3563, -0.7140],
        [ 0.3660, -0.2286],
        [ 0.5619,  0.0262],
        [-0.4208, -0.4772],
        [-0.7404, -0.5157],
        [-1.4992,  0.4404]])

In [15]:
contribution_logits_ps.mean(0)


tensor([[-0.6563, -0.6448],
        [ 0.0251, -0.5451],
        [ 0.7821, -0.9319],
        [-0.4028, -0.7478],
        [-0.8475, -0.8607],
        [-0.3234, -0.2592]])